In [29]:
from pyspark.sql.functions import *

StatementMeta(vnpaypool3, 18, 30, Finished, Available, Finished)

In [30]:
#Set the folder paths so that it can be used later. 
bronze_folder_path = 'abfss://bronze@vnpayproject.dfs.core.windows.net/'
silver_folder_path = 'abfss://silver@vnpayproject.dfs.core.windows.net/'
gold_folder_path = 'abfss://gold@vnpayproject.dfs.core.windows.net/'

StatementMeta(vnpaypool3, 18, 31, Finished, Available, Finished)

In [31]:
# Read the table into a DataFrame
df = spark.table("vnpay_silver_spark.alltransaction_fact")

StatementMeta(vnpaypool3, 18, 32, Finished, Available, Finished)

In [32]:
display(df.limit(5))

StatementMeta(vnpaypool3, 18, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8ac59ee4-0cd6-44c0-a698-abcc0954a16d)

In [33]:
new_df = df.select(
    col("ma_gd").alias("txn_id"),
    col("ten_merchant").alias("merchant"),
    col("master_mc").alias("master_merchant"),
    col("ma_ten_terminal").alias("terminal"),
    col("kenh_thanh_toan").alias("payment_method"),
    col("ten_kh_thanh_toan").alias("user_name"),
    col("so_dien_thoai").alias("phone"),
    col("don_vi_tt").alias("issuer"),
    col("so_tai_khoan").alias("account_number"),
    col("loai_the_tai_khoan").alias("account_type"),
    col("so_tien_truoc_km").alias("org_amount"),
    col("so_tien_sau_km").alias("final_amount"),
    (col("so_tien_truoc_km") - col("so_tien_sau_km")).alias("promotion_amount"),
    col("thoi_gian_thanh_toan").alias("payment_time"),
    to_date(col("thoi_gian_thanh_toan")).alias("payment_date"),
    col("year"),
    col("month"),
    col("trang_thai").alias("payment_status"),
    col("mcc_noi_dia").alias("domestic_mcc"),
    col("mcc_quoc_te").alias("int_mcc"),
    col("user_id"),
    col("terminal_id"),
    col("issuer_id"),
    col("merchant_id"),
    col("master_merchant_id")
).filter((col("year").isNotNull()) & (col("year") != 2006))

display(new_df.limit(10))

StatementMeta(vnpaypool3, 18, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 404306af-85f5-4f57-9822-588935ca22ac)

In [34]:
# Write to Delta Lake with partition by year and month
new_df.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save(f"{gold_folder_path}/alltransaction_fact")

StatementMeta(vnpaypool3, 18, 35, Finished, Available, Finished)

In [35]:
# Create External Table pointing to existing Delta Lake location
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS vnpay_gold_spark.alltransaction_fact
    USING DELTA
    LOCATION '{gold_folder_path}/alltransaction_fact'
""")

StatementMeta(vnpaypool3, 18, 36, Finished, Available, Finished)

DataFrame[]